In [3]:
import os
os.chdir("/home/jovyan/work")

from neomodel import BooleanProperty, IntegerProperty, RelationshipTo, StructuredNode, StringProperty
import pandas as pd
from tqdm import tqdm

from src.models import Game, JumpBall, Player, Turnover, Violation
from src.utils import init_connection

In [ ]:
init_connection()

### JumpBall

In [ ]:
df = pd.read_csv("./data/play_by_play_files/play_by_play_jumpball.csv")

In [ ]:
df.shape[0]

In [ ]:
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    jb = JumpBall()
    jb.name = row["name"]
    jb.won_by = row["wonBy"]
    jb.elapsed_seconds = row["secondsElapsed"]
    jb.quarter = row["quarter"]
    jb.save()
    
    jb.away_player.connect(Player.nodes.get(msfID=row["awayPlayerID"]))
    jb.home_player.connect(Player.nodes.get(msfID=row["homePlayerID"]))
    jb.game.connect(Game.nodes.get(game_id=row["gameID"]))

### Violation

In [ ]:
df = pd.read_csv("./data/play_by_play_files/play_by_play_violation.csv")

In [ ]:
for _, row in tqdm(df.iterrows(), df.shape[0]):
    v = Violation()
    v.name = row["name"]
    v.team_abbreviation = row["teamAbbreviation"]
    v.vioalation_type = row["teamOrPersonal"]
    v.elapsed_seconds = row["secondsElapsed"]
    v.quarter = row["quarter"]
    v.save()
    
    v.game.connect(Game.nodes.get(game_id=row["gameID"]))
    if v.vioalation_type == "PERSONAL":
        v.player.connect(Player.nodes.get(msfID=row["playerID"]))

### Turnover

In [ ]:
df = pd.read_csv("./data/play_by_play_files/play_by_play_turnover.csv")

In [ ]:
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    t = Turnover()
    t.name = row["name"]
    t.is_stolen = row["isStolen"]
    t.turnover_type = row["type"]
    t.elapsed_seconds = row["secondsElapsed"]
    t.quarter = row["quarter"]
    t.save()
    
    t.lost_by_player.connect(Player.nodes.get(msfID=row["lostByPlayerID"]))
    t.game.connect(Game.nodes.get(game_id=row["gameID"]))
    if t.is_stolen:
        t.stolen_by_player.connect(Player.nodes.get(msfID=row["stolenByPlayerID"]))

In [4]:
for f in os.listdir("./data/play_by_play_files/"):
    size = pd.read_csv("./data/play_by_play_files/{f}".format(**locals())).shape[0]
    print("{f} : {size}".format(**locals()))

play_by_play_jumpball.csv : 6551
play_by_play_violation.csv : 6263
play_by_play_turnover.csv : 104710
play_by_play_foul.csv : 159372
play_by_play_fieldGoalAttempt.csv : 657302
play_by_play_rebound.csv : 398604
play_by_play_substitution.csv : 448210
play_by_play_freeThrowAttempt.csv : 175476
